# Face detection 

_The face detection algorithms detects facial pattern through varios classifiers in an image to detect if there is a face in it_. In this tutorial, we shall see the following most popular algorithms on Face Detaction and apply them through python.
1. __Haarcascade Classifier__: This the most simple face ddetection algorithm for detecting Face and Eyes. This classifier along with detecting faces, can also detect other objects like a human body or cars. 
2. __Histogram of Oriented Gradient (HOG)__: HOG is more effiecient that Haarcascade. 
3. __Convolutional Neural Network (CNN)__: CNN is aguably the most robust algorithm for face and object detection.   

In the implementation, we shall use OpenCV for the Haarcascade classifier and Dlib library for HOG and CNN. Note that, we're not going to build the classifier in this Lab, rather we shall see how to leverage existinig classifiers for detecting objects. Building classifiers fall  under the discussion of Feature Training which will be covered during upcoming sections of this course.

## brief of Haarcasde classifier

* Every image is comprised by a grid of pixels. The shape of the grid in $m\times n$ called the resolution and the number of bits represent each pixel is called the bit-depth or color-depth. 
* In face detection it is a common practice to convert a color image into a grey-scale image; because, a greay scale image reduces the image size by 67% having preserved all the patterns for detection. 
* Steps
    1. The classifier is trained with two sets of objects one with faces and the other expect face. 
    2. The training samples then gets fed into AdaBoost training algorithm for computing the classifiers by the facial features.
    3. An passes through all the classifiers, if all the features are detected then Haarcaascade detects a face in it. The number of features passed determines the confifence value. If the confidence value does not exceed a cirtain theshold, the algorithm does not detect an face. 
    4. The classifier segments an image into multiple sub-areas and applies step 3 on each segment. When a Face is found at any segment the location of the segment is returned.




In [2]:
import cv2 as cv    #import the openCV library

__Step 1__: Load an image from local directory

In [3]:
image = cv.imread('images/people2.jpg')    # read the image 
cv.imshow(winname='original', mat=image)   # show the image 
cv.waitKey(0)

-1

![](figs/fd_1.png)

First lets see the image resolution

In [49]:
height, width, depth = image.shape
print(f'Resolution is {height} X {width} with {depth} color channels')

Resolution is 600 X 800 with 3 color channels


__Step 2__: downscale the image for larger image 

In [51]:
scale_factor = 0.75     # set the scale factor 
if scale_factor == 1:
    image_scalled = image
else:
    image_scalled = cv.resize(image,                                                   # source image 
                            (int(width * scale_factor), int(height * scale_factor)),  # target resolution 
                            interpolation=cv.INTER_AREA)                              # interpolation used
cv.imshow(winname='Scalled', mat=image_scalled)   # show the image 
cv.waitKey(0)                     

-1

![](figs/fd2.png)

__Step 3:__  Convert the image into a Greayscale image

In [52]:
image_grey = cv.cvtColor(image_scalled,cv.COLOR_BGR2GRAY)      # converting into a grey scale
cv.imshow(winname='Greyscaled', mat=image_grey)   # show the image 
cv.waitKey(0)   

#print the image resolution   
height, width = image_grey.shape
print(f'Resolution is {height} X {width}')

Resolution is 450 X 600


![](figs/fd_3.png)

__Step 4:__ Creatinig a face detector and detect faces
The pretained classifiers can be found at : https://github.com/opencv/opencv/tree/master/data/haarcascades. The files are located locally in the `classifier/haarcascade` folder. 

In [53]:
## 4.1. Load the classifier 
face_detector = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")

In [55]:
# 4.2. Detect faces 
face_detected = face_detector.detectMultiScale(image_grey, 
                                              scaleFactor = 1.1, # scale according to the face size (>=1) 
                                              minNeighbors = 5,  # reduces False positives 
                                              minSize = (10,10)) # min size of a face to be detected
                                               
print(face_detected)   # returns a list of [x, y, width, height] of the detected area 

[[231 275 150 150]
 [158 174 119 119]
 [330  35 118 118]
 [ 41 150 151 151]
 [222  54 121 121]
 [349 257 180 180]]


__Step 5:__ Draw reactangle around the face 

In [23]:
temp = None
temp = image_scalled.copy()   # duplicate the image
for x,y,w,h in face_detected:
    # determine point1 and point2
    top_left = (x,y)
    bottom_right = (x+h, y+w)
    image_marked = cv.rectangle(temp, pt1=top_left, pt2=bottom_right, color=(0,255,0),thickness=2) #draw rect

cv.imshow('Face_detected', image_marked)
cv.waitKey(0)
cv.destroyAllWindows()

![](figs/fd_4.png)

# Face Detection from a video 

In [4]:
import cv2 as cv
def face_detect(frame):
    '''
    Input : Frame 
    output: Frame with face detected 
    '''
    frame_gr = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    face_detector = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
    face_detected = face_detector.detectMultiScale(frame_gr)
    number_of_faces = len(face_detected.tolist())
    count = number_of_faces
    if count:
        for x,y,w,h in face_detected:
            cv.rectangle(img=frame, pt1=(x,y), pt2=(x+h,y+h), color=(255,0,255),thickness=2)
            cv.putText(img=frame,text=f'Face {count}', org=(x,y),fontFace=cv.FONT_HERSHEY_PLAIN, fontScale=1, color=(0,0,255))
            count-=1
    
    cv.putText(img=frame,text=f'Face Count = {number_of_faces}', org=(100,100),fontFace=cv.FONT_HERSHEY_PLAIN, fontScale=2, color=(0,0,255))
    return frame

def read_video(file):
    capture = cv.VideoCapture(file)
    while True:
        isTrue, frame = capture.read()
        frame_org = frame.copy()
        frame_marked = face_detect(frame)
        cv.imshow(winname='Original', mat=frame_org)
        cv.imshow(winname='Face Detected', mat=frame_marked)
        if cv.waitKey(20) & 0xFF == ord('d'):        # stop the video is the key 'd' is pressed (you can change as per your choice)
            break
    capture.release()  
    cv.destroyAllWindows()

In [5]:
read_video(file='videos\sample_vid3.mp4')

AttributeError: 'tuple' object has no attribute 'tolist'